# Counterfactuals: orcs
## 8 characteristics

## Prepare data

In [1]:
from training.creating_dataset import load_data, preprocess_data

In [2]:
import pandas as pd
import sys

import os
import pathlib

sys.path.append("../../..")

current_path = os.getcwd()
current_path = os.getcwd()
DATASETS_DIR = pathlib.Path(current_path).parent.parent.parent / "pathfinder_2e_data"
DATASET_FILES = [
    "pathfinder-bestiary.db",
    "pathfinder-bestiary-2.db",
    "pathfinder-bestiary-3.db",
]
DATASET_PATHS = [f"{DATASETS_DIR}/{file}" for file in DATASET_FILES]

df = load_data(DATASET_PATHS)

In [3]:
# df = df[ORDERED_CHARACTERISTICS + ["book", "level"]]
df.head()

,_id,img,items,name,type,system.abilities.cha.mod,system.abilities.con.mod,system.abilities.dex.mod,system.abilities.int.mod,system.abilities.str.mod,...,system.traits.size.value,system.traits.value,system.schema.version,system.schema.lastMigration,flags.core.sourceId,system.attributes.resistances,system.traits.attitude.value,prototypeToken.name,system.attributes.hardness.value,system.attributes.adjustment
0,024PqcF8yMRBrPuq,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...",Adult White Dragon,npc,1,5,2,1,7,...,lg,"[cold, dragon]",0.827,None,Compendium.pf2e.pathfinder-bestiary.024PqcF8yM...,NaN,NaN,NaN,NaN,NaN
1,05E3kkjoLZVjFOeO,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'mYNDkS27YQRyy1F0', 'img': 'systems/p...",Brontosaurus,npc,1,5,0,-4,9,...,grg,"[animal, dinosaur]",0.827,None,Compendium.pf2e.pathfinder-bestiary.05E3kkjoLZ...,NaN,NaN,NaN,NaN,NaN
2,05wwpHHsBlxBbdkN,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'WkPeg600zGONsuJz', 'img': 'systems/p...",Giant Anaconda,npc,-2,6,3,-4,7,...,huge,[animal],0.827,None,Compendium.pf2e.pathfinder-bestiary.05wwpHHsBl...,NaN,NaN,NaN,NaN,NaN
3,0FGz2eXm0SB04sJW,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'joBYS96mXSnZC1WB', 'img': 'systems/p...",Ancient Green Dragon,npc,6,5,4,6,7,...,grg,"[amphibious, dragon]",0.827,None,Compendium.pf2e.pathfinder-bestiary.0FGz2eXm0S...,NaN,NaN,NaN,NaN,NaN
4,0HjVFx8qIKDCfblg,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'xlUnQcjrxeMXu5Cd', 'img': 'systems/p...",Lantern Archon,npc,1,1,3,-1,-5,...,sm,"[archon, celestial]",0.827,None,Compendium.pf2e.pathfinder-bestiary.0HjVFx8qIK...,"[{'type': 'fire', 'value': 3}]",NaN,NaN,NaN,NaN


In [4]:
df_orcs = df[df.name.str.contains("Orc ")]

In [5]:
df_orcs.name

240    Orc Warchief
284       Orc Brute
323     Orc Warrior
Name: name, dtype: object

In [6]:
orcs_ids_names = {df_orcs.loc[idx]["name"]: idx for idx in df_orcs.index}
orcs_ids_names

{'Orc Warchief': 240, 'Orc Brute': 284, 'Orc Warrior': 323}

In [7]:
from serving.backend.constants import ORDERED_CHARACTERISTICS


orcs = preprocess_data(df_orcs, ORDERED_CHARACTERISTICS)
orcs

,hp,str,con,ac,book,level,wis,int,dex,cha
240,32,4,1,19,Pathfinder Bestiary,2,1,-1,2,2
284,15,3,3,15,Pathfinder Bestiary,0,1,-1,2,0
323,23,4,3,18,Pathfinder Bestiary,1,1,-1,2,0


In [8]:
orcs = orcs[ORDERED_CHARACTERISTICS + ["level"]]

In [9]:
orcs

,str,dex,con,int,wis,cha,ac,hp,level
240,4,2,1,-1,1,2,19,32,2
284,3,2,3,-1,1,0,15,15,0
323,4,2,3,-1,1,0,18,23,1


In [10]:
dataset = preprocess_data(df, ORDERED_CHARACTERISTICS)
dataset = dataset[ORDERED_CHARACTERISTICS + ["level"]]

In [11]:
dataset.head()

,str,dex,con,int,wis,cha,ac,hp,level
0,7,2,5,1,2,1,29,215,10
1,9,0,5,-4,2,1,28,220,10
2,7,3,6,-4,3,-2,25,175,8
3,7,4,5,6,5,6,41,315,17
4,-5,3,1,-1,1,1,16,20,1


In [13]:
import joblib
from training.create_model import get_fitted_model


model = joblib.load(filename="../../../saved_models/current_model.pkl")

# model = get_fitted_model(classifier_name="lightgbm", X_train=dataset[ORDERED_CHARACTERISTICS], y_train=dataset["level"])

## Counterfactuals for orcs

In [14]:
from serving.backend.generate_counterfactuals import generate_counterfactuals

C:\Users\jolka\OneDrive\Dokumenty\Inzynierka\ML-for-TTRPG-game-design\venv\lib\site-packages\dice_ml\utils\exception.py:12: UserWarning: UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.
  warnings.warn("UserConfigValidationException will be deprecated from dice_ml.utils. "


In [15]:
orc_brute = orcs.loc[orcs_ids_names["Orc Brute"]].to_dict()
orc_warrior = orcs.loc[orcs_ids_names["Orc Warrior"]].to_dict()
orc_warchief = orcs.loc[orcs_ids_names["Orc Warchief"]].to_dict()

### Orc Brute

Compare to others (levels 1 and 2)

In [16]:
result = generate_counterfactuals(
    monster_stats=orc_brute, model=model, new_level=1, df=dataset, total_cf=15
)

100%|██████████| 1/1 [01:30<00:00, 90.57s/it]


In [17]:
diff_columns = [f"{col}_diff" for col in ORDERED_CHARACTERISTICS]


def compare_monsters(cfs, compare_to_monster):
    compare_df = pd.DataFrame(columns=ORDERED_CHARACTERISTICS)
    diff_df = pd.DataFrame(columns=diff_columns)

    for cf in cfs:
        cfs_comp_dict = dict()
        diff_dict = dict()
        for i, col in enumerate(ORDERED_CHARACTERISTICS):
            val = cf[i]
            cfs_comp_dict[col] = [val] if val != compare_to_monster[col] else ["-"]
            diff_dict[f"{col}_diff"] = (
                [val - compare_to_monster[col]]
                if val != compare_to_monster[col]
                else ["-"]
            )

        comparison = pd.DataFrame.from_dict(cfs_comp_dict)
        difference = pd.DataFrame.from_dict(diff_dict)
        compare_df = pd.concat([comparison, compare_df.loc[:]]).reset_index(drop=True)
        diff_df = pd.concat([difference, diff_df]).reset_index(drop=True)

    return compare_df, diff_df

##### Compare to original data

In [18]:
comp, diff = compare_monsters(result["values"], orc_brute)
comp

,str,dex,con,int,wis,cha,ac,hp
0,2.0,-,-,1.0,4.0,3.0,-,25.0
1,-,-2.0,-,-3.0,-0.0,-,13.0,62.0
2,4.0,-,4.0,-4.0,3.0,-,16.0,-
3,-,4.0,1.0,-,-,1.0,17.0,19.0
4,-,3.0,-,-4.0,-,-2.0,16.0,20.0
5,-,-,2.0,-4.0,-,-1.0,17.0,17.0
6,4.0,-,2.0,0.0,-,1.0,-,24.0
7,0.0,-5.0,-,-4.0,-,-,-,30.0
8,0.0,4.0,-,-4.0,-,-,-,30.0
9,-,4.0,-,-4.0,-,-,16.0,21.0


In [19]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-1.0,-,-,2.0,3.0,3.0,-,10.0
1,-,-4.0,-,-2.0,-1.0,-,-2.0,47.0
2,1.0,-,1.0,-3.0,2.0,-,1.0,-
3,-,2.0,-2.0,-,-,1.0,2.0,4.0
4,-,1.0,-,-3.0,-,-2.0,1.0,5.0
5,-,-,-1.0,-3.0,-,-1.0,2.0,2.0
6,1.0,-,-1.0,1.0,-,1.0,-,9.0
7,-3.0,-7.0,-,-3.0,-,-,-,15.0
8,-3.0,2.0,-,-3.0,-,-,-,15.0
9,-,2.0,-,-3.0,-,-,1.0,6.0


##### Compare to *Orc Warrior*

In [20]:
comp, diff = compare_monsters(result["values"], orc_warrior)

In [21]:
comp

,str,dex,con,int,wis,cha,ac,hp
0,2.0,-,-,1.0,4.0,3.0,15.0,25.0
1,3.0,-2.0,-,-3.0,-0.0,-,13.0,62.0
2,-,-,4.0,-4.0,3.0,-,16.0,15.0
3,3.0,4.0,1.0,-,-,1.0,17.0,19.0
4,3.0,3.0,-,-4.0,-,-2.0,16.0,20.0
5,3.0,-,2.0,-4.0,-,-1.0,17.0,17.0
6,-,-,2.0,0.0,-,1.0,15.0,24.0
7,0.0,-5.0,-,-4.0,-,-,15.0,30.0
8,0.0,4.0,-,-4.0,-,-,15.0,30.0
9,3.0,4.0,-,-4.0,-,-,16.0,21.0


In [22]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-2.0,-,-,2.0,3.0,3.0,-3.0,2.0
1,-1.0,-4.0,-,-2.0,-1.0,-,-5.0,39.0
2,-,-,1.0,-3.0,2.0,-,-2.0,-8.0
3,-1.0,2.0,-2.0,-,-,1.0,-1.0,-4.0
4,-1.0,1.0,-,-3.0,-,-2.0,-2.0,-3.0
5,-1.0,-,-1.0,-3.0,-,-1.0,-1.0,-6.0
6,-,-,-1.0,1.0,-,1.0,-3.0,1.0
7,-4.0,-7.0,-,-3.0,-,-,-3.0,7.0
8,-4.0,2.0,-,-3.0,-,-,-3.0,7.0
9,-1.0,2.0,-,-3.0,-,-,-2.0,-2.0


In [23]:
result = generate_counterfactuals(
    monster_stats=orc_brute, model=model, new_level=2, df=dataset, total_cf=10
)

100%|██████████| 1/1 [00:13<00:00, 13.90s/it]


##### Compere to original data

In [24]:
comp, diff = compare_monsters(result["values"], orc_brute)
comp

,str,dex,con,int,wis,cha,ac,hp
0,-,4.0,-,2.0,-,1.0,18.0,30.0
1,4.0,3.0,2.0,-,-,-,18.0,29.0
2,4.0,-,-,-4.0,2.0,-,17.0,35.0
3,4.0,-,-,0.0,-,-1.0,18.0,34.0
4,-,-,4.0,-,3.0,-,18.0,30.0
5,-,4.0,2.0,-,-,-,19.0,28.0
6,-,-,-,-4.0,-,-2.0,18.0,32.0
7,-,-,-,-4.0,-,-2.0,18.0,30.0
8,4.0,-,-,-,-,1.0,17.0,35.0
9,4.0,-,-,-,-,-,17.0,34.0


In [25]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-,2.0,-,3.0,-,1.0,3.0,15.0
1,1.0,1.0,-1.0,-,-,-,3.0,14.0
2,1.0,-,-,-3.0,1.0,-,2.0,20.0
3,1.0,-,-,1.0,-,-1.0,3.0,19.0
4,-,-,1.0,-,2.0,-,3.0,15.0
5,-,2.0,-1.0,-,-,-,4.0,13.0
6,-,-,-,-3.0,-,-2.0,3.0,17.0
7,-,-,-,-3.0,-,-2.0,3.0,15.0
8,1.0,-,-,-,-,1.0,2.0,20.0
9,1.0,-,-,-,-,-,2.0,19.0


##### Compare to *Orc Warchief*

In [55]:
comp, diff = compare_monsters(result["values"], orc_warchief)

In [27]:
comp

,str,dex,con,int,wis,cha,ac,hp
0,3.0,4.0,3.0,2.0,-,1.0,18.0,30.0
1,-,3.0,2.0,-,-,0.0,18.0,29.0
2,-,-,3.0,-4.0,2.0,0.0,17.0,35.0
3,-,-,3.0,0.0,-,-1.0,18.0,34.0
4,3.0,-,4.0,-,3.0,0.0,18.0,30.0
5,3.0,4.0,2.0,-,-,0.0,-,28.0
6,3.0,-,3.0,-4.0,-,-2.0,18.0,-
7,3.0,-,3.0,-4.0,-,-2.0,18.0,30.0
8,-,-,3.0,-,-,1.0,17.0,35.0
9,-,-,3.0,-,-,0.0,17.0,34.0


In [28]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-1.0,2.0,2.0,3.0,-,-1.0,-1.0,-2.0
1,-,1.0,1.0,-,-,-2.0,-1.0,-3.0
2,-,-,2.0,-3.0,1.0,-2.0,-2.0,3.0
3,-,-,2.0,1.0,-,-3.0,-1.0,2.0
4,-1.0,-,3.0,-,2.0,-2.0,-1.0,-2.0
5,-1.0,2.0,1.0,-,-,-2.0,-,-4.0
6,-1.0,-,2.0,-3.0,-,-4.0,-1.0,-
7,-1.0,-,2.0,-3.0,-,-4.0,-1.0,-2.0
8,-,-,2.0,-,-,-1.0,-2.0,3.0
9,-,-,2.0,-,-,-2.0,-2.0,2.0


### Orc Warrior

In [29]:
result = generate_counterfactuals(
    monster_stats=orc_warrior, model=model, new_level=0, df=dataset, total_cf=10
)

100%|██████████| 1/1 [07:46<00:00, 466.34s/it]


##### Compare to original data

In [30]:
comp, diff = compare_monsters(result["values"], orc_warrior)
comp

,str,dex,con,int,wis,cha,ac,hp
0,3.0,-,2.0,-5.0,2.0,-,16.0,15.0
1,1.0,1.0,-,-2.0,-,-2.0,13.0,21.0
2,2.0,1.0,-,0.0,2.0,-,16.0,15.0
3,-,-2.0,6.0,-2.0,-,-2.0,-,5.0
4,-0.0,-,-,-5.0,-2.0,-,11.0,12.0
5,-0.0,-,5.0,-5.0,-,-,11.0,7.0
6,3.0,-,-,-4.0,2.0,-,14.0,16.0
7,2.0,-,-,0.0,2.0,-,14.0,20.0
8,3.0,-,-,-4.0,-,1.0,15.0,18.0
9,3.0,-,-,-,-,-,15.0,15.0


In [31]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-1.0,-,-1.0,-4.0,1.0,-,-2.0,-8.0
1,-3.0,-1.0,-,-1.0,-,-2.0,-5.0,-2.0
2,-2.0,-1.0,-,1.0,1.0,-,-2.0,-8.0
3,-,-4.0,3.0,-1.0,-,-2.0,-,-18.0
4,-4.0,-,-,-4.0,-3.0,-,-7.0,-11.0
5,-4.0,-,2.0,-4.0,-,-,-7.0,-16.0
6,-1.0,-,-,-3.0,1.0,-,-4.0,-7.0
7,-2.0,-,-,1.0,1.0,-,-4.0,-3.0
8,-1.0,-,-,-3.0,-,1.0,-3.0,-5.0
9,-1.0,-,-,-,-,-,-3.0,-8.0


##### Compare to *Orc brute*

In [32]:
comp, diff = compare_monsters(result["values"], orc_brute)
comp

,str,dex,con,int,wis,cha,ac,hp
0,-,-,2.0,-5.0,2.0,-,16.0,-
1,1.0,1.0,-,-2.0,-,-2.0,13.0,21.0
2,2.0,1.0,-,0.0,2.0,-,16.0,-
3,4.0,-2.0,6.0,-2.0,-,-2.0,18.0,5.0
4,-0.0,-,-,-5.0,-2.0,-,11.0,12.0
5,-0.0,-,5.0,-5.0,-,-,11.0,7.0
6,-,-,-,-4.0,2.0,-,14.0,16.0
7,2.0,-,-,0.0,2.0,-,14.0,20.0
8,-,-,-,-4.0,-,1.0,-,18.0
9,-,-,-,-,-,-,-,-


In [36]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-,-,-1.0,-4.0,1.0,-,1.0,-
1,-2.0,-1.0,-,-1.0,-,-2.0,-2.0,6.0
2,-1.0,-1.0,-,1.0,1.0,-,1.0,-
3,1.0,-4.0,3.0,-1.0,-,-2.0,3.0,-10.0
4,-3.0,-,-,-4.0,-3.0,-,-4.0,-3.0
5,-3.0,-,2.0,-4.0,-,-,-4.0,-8.0
6,-,-,-,-3.0,1.0,-,-1.0,1.0
7,-1.0,-,-,1.0,1.0,-,-1.0,5.0
8,-,-,-,-3.0,-,1.0,-,3.0
9,-,-,-,-,-,-,-,-


In [37]:
result = generate_counterfactuals(
    monster_stats=orc_warrior, model=model, new_level=2, df=dataset, total_cf=10
)

100%|██████████| 1/1 [00:09<00:00,  9.84s/it]


##### Compare to original data

In [38]:
comp, diff = compare_monsters(result["values"], orc_warrior)
comp

,str,dex,con,int,wis,cha,ac,hp
0,-,-,4.0,-4.0,2.0,-,-,32.0
1,-,1.0,-,-3.0,2.0,-,-,30.0
2,-,1.0,-,-,2.0,1.0,-,28.0
3,3.0,-,-,-4.0,-,-2.0,-,32.0
4,3.0,-,-,-4.0,-,-2.0,-,30.0
5,-,-,-,-4.0,2.0,-,17.0,35.0
6,-,3.0,2.0,-,-,-,-,29.0
7,-,-,-,0.0,-,-1.0,-,34.0
8,-,-,-,-,-,1.0,17.0,35.0
9,-,-,-,-,-,-,17.0,34.0


In [39]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-,-,1.0,-3.0,1.0,-,-,9.0
1,-,-1.0,-,-2.0,1.0,-,-,7.0
2,-,-1.0,-,-,1.0,1.0,-,5.0
3,-1.0,-,-,-3.0,-,-2.0,-,9.0
4,-1.0,-,-,-3.0,-,-2.0,-,7.0
5,-,-,-,-3.0,1.0,-,-1.0,12.0
6,-,1.0,-1.0,-,-,-,-,6.0
7,-,-,-,1.0,-,-1.0,-,11.0
8,-,-,-,-,-,1.0,-1.0,12.0
9,-,-,-,-,-,-,-1.0,11.0


##### Compare to *Orc Warchief*

In [40]:
comp, diff = compare_monsters(result["values"], orc_warchief)

In [41]:
comp

,str,dex,con,int,wis,cha,ac,hp
0,-,-,4.0,-4.0,2.0,0.0,18.0,-
1,-,1.0,3.0,-3.0,2.0,0.0,18.0,30.0
2,-,1.0,3.0,-,2.0,1.0,18.0,28.0
3,3.0,-,3.0,-4.0,-,-2.0,18.0,-
4,3.0,-,3.0,-4.0,-,-2.0,18.0,30.0
5,-,-,3.0,-4.0,2.0,0.0,17.0,35.0
6,-,3.0,2.0,-,-,0.0,18.0,29.0
7,-,-,3.0,0.0,-,-1.0,18.0,34.0
8,-,-,3.0,-,-,1.0,17.0,35.0
9,-,-,3.0,-,-,0.0,17.0,34.0


In [42]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-,-,3.0,-3.0,1.0,-2.0,-1.0,-
1,-,-1.0,2.0,-2.0,1.0,-2.0,-1.0,-2.0
2,-,-1.0,2.0,-,1.0,-1.0,-1.0,-4.0
3,-1.0,-,2.0,-3.0,-,-4.0,-1.0,-
4,-1.0,-,2.0,-3.0,-,-4.0,-1.0,-2.0
5,-,-,2.0,-3.0,1.0,-2.0,-2.0,3.0
6,-,1.0,1.0,-,-,-2.0,-1.0,-3.0
7,-,-,2.0,1.0,-,-3.0,-1.0,2.0
8,-,-,2.0,-,-,-1.0,-2.0,3.0
9,-,-,2.0,-,-,-2.0,-2.0,2.0


### Orc Warchief

In [43]:
result = generate_counterfactuals(
    monster_stats=orc_warchief, model=model, new_level=0, df=dataset, total_cf=10
)

100%|██████████| 1/1 [08:36<00:00, 516.06s/it]


##### Compare to original data

In [44]:
comp, diff = compare_monsters(result["values"], orc_warchief)
comp

,str,dex,con,int,wis,cha,ac,hp
0,-2.0,-,-,-4.0,3.0,0.0,16.0,15.0
1,1.0,3.0,-,-4.0,-,-4.0,16.0,15.0
2,0.0,3.0,-,-4.0,-,-2.0,16.0,15.0
3,3.0,-,3.0,-4.0,-,1.0,15.0,18.0
4,2.0,1.0,-,-,2.0,3.0,15.0,13.0
5,-0.0,-,4.0,6.0,-,-,15.0,9.0
6,-0.0,-,4.0,-5.0,-,-,15.0,15.0
7,-2.0,-,-,-2.0,-,9.0,13.0,5.0
8,3.0,-,3.0,-,-,0.0,15.0,15.0
9,-1.0,-,0.0,0.0,-,-,17.0,4.0


In [45]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-6.0,-,-,-3.0,2.0,-2.0,-3.0,-17.0
1,-3.0,1.0,-,-3.0,-,-6.0,-3.0,-17.0
2,-4.0,1.0,-,-3.0,-,-4.0,-3.0,-17.0
3,-1.0,-,2.0,-3.0,-,-1.0,-4.0,-14.0
4,-2.0,-1.0,-,-,1.0,1.0,-4.0,-19.0
5,-4.0,-,3.0,7.0,-,-,-4.0,-23.0
6,-4.0,-,3.0,-4.0,-,-,-4.0,-17.0
7,-6.0,-,-,-1.0,-,7.0,-6.0,-27.0
8,-1.0,-,2.0,-,-,-2.0,-4.0,-17.0
9,-5.0,-,-1.0,1.0,-,-,-2.0,-28.0


##### Compare to *Orc brute*

In [46]:
comp, diff = compare_monsters(result["values"], orc_brute)

In [47]:
comp

,str,dex,con,int,wis,cha,ac,hp
0,-2.0,-,1.0,-4.0,3.0,-,16.0,-
1,1.0,3.0,1.0,-4.0,-,-4.0,16.0,-
2,0.0,3.0,1.0,-4.0,-,-2.0,16.0,-
3,-,-,-,-4.0,-,1.0,-,18.0
4,2.0,1.0,1.0,-,2.0,3.0,-,13.0
5,-0.0,-,4.0,6.0,-,2.0,-,9.0
6,-0.0,-,4.0,-5.0,-,2.0,-,-
7,-2.0,-,1.0,-2.0,-,9.0,13.0,5.0
8,-,-,-,-,-,-,-,-
9,-1.0,-,0.0,0.0,-,2.0,17.0,4.0


In [48]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-5.0,-,-2.0,-3.0,2.0,-,1.0,-
1,-2.0,1.0,-2.0,-3.0,-,-4.0,1.0,-
2,-3.0,1.0,-2.0,-3.0,-,-2.0,1.0,-
3,-,-,-,-3.0,-,1.0,-,3.0
4,-1.0,-1.0,-2.0,-,1.0,3.0,-,-2.0
5,-3.0,-,1.0,7.0,-,2.0,-,-6.0
6,-3.0,-,1.0,-4.0,-,2.0,-,-
7,-5.0,-,-2.0,-1.0,-,9.0,-2.0,-10.0
8,-,-,-,-,-,-,-,-
9,-4.0,-,-3.0,1.0,-,2.0,2.0,-11.0


In [49]:
result = generate_counterfactuals(
    monster_stats=orc_warchief, model=model, new_level=1, df=dataset, total_cf=10
)

100%|██████████| 1/1 [00:28<00:00, 28.24s/it]


##### Compare to original data

In [50]:
comp, diff = compare_monsters(result["values"], orc_warchief)
comp

,str,dex,con,int,wis,cha,ac,hp
0,0.0,3.0,-,0.0,-,4.0,16.0,25.0
1,3.0,3.0,2.0,-,0.0,-,17.0,18.0
2,2.0,4.0,-,0.0,-,1.0,18.0,19.0
3,-,3.0,4.0,-5.0,-,-,16.0,25.0
4,-5.0,3.0,-,-,-,1.0,16.0,20.0
5,0.0,3.0,2.0,-,-,-,16.0,22.0
6,3.0,4.0,-,-,-,1.0,17.0,19.0
7,-,-,2.0,0.0,-,1.0,15.0,24.0
8,-,-,3.0,-,-,0.0,16.0,21.0
9,-,-,3.0,-,-,0.0,18.0,23.0


In [51]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-4.0,1.0,-,1.0,-,2.0,-3.0,-7.0
1,-1.0,1.0,1.0,-,-1.0,-,-2.0,-14.0
2,-2.0,2.0,-,1.0,-,-1.0,-1.0,-13.0
3,-,1.0,3.0,-4.0,-,-,-3.0,-7.0
4,-9.0,1.0,-,-,-,-1.0,-3.0,-12.0
5,-4.0,1.0,1.0,-,-,-,-3.0,-10.0
6,-1.0,2.0,-,-,-,-1.0,-2.0,-13.0
7,-,-,1.0,1.0,-,-1.0,-4.0,-8.0
8,-,-,2.0,-,-,-2.0,-3.0,-11.0
9,-,-,2.0,-,-,-2.0,-1.0,-9.0


##### Compare to *Orc warrior*

In [52]:
comp, diff = compare_monsters(result["values"], orc_warrior)

In [53]:
comp

,str,dex,con,int,wis,cha,ac,hp
0,0.0,3.0,1.0,0.0,-,4.0,16.0,25.0
1,3.0,3.0,2.0,-,0.0,2.0,17.0,18.0
2,2.0,4.0,1.0,0.0,-,1.0,-,19.0
3,-,3.0,4.0,-5.0,-,2.0,16.0,25.0
4,-5.0,3.0,1.0,-,-,1.0,16.0,20.0
5,0.0,3.0,2.0,-,-,2.0,16.0,22.0
6,3.0,4.0,1.0,-,-,1.0,17.0,19.0
7,-,-,2.0,0.0,-,1.0,15.0,24.0
8,-,-,-,-,-,-,16.0,21.0
9,-,-,-,-,-,-,-,-


In [54]:
diff

,str_diff,dex_diff,con_diff,int_diff,wis_diff,cha_diff,ac_diff,hp_diff
0,-4.0,1.0,-2.0,1.0,-,4.0,-2.0,2.0
1,-1.0,1.0,-1.0,-,-1.0,2.0,-1.0,-5.0
2,-2.0,2.0,-2.0,1.0,-,1.0,-,-4.0
3,-,1.0,1.0,-4.0,-,2.0,-2.0,2.0
4,-9.0,1.0,-2.0,-,-,1.0,-2.0,-3.0
5,-4.0,1.0,-1.0,-,-,2.0,-2.0,-1.0
6,-1.0,2.0,-2.0,-,-,1.0,-1.0,-4.0
7,-,-,-1.0,1.0,-,1.0,-3.0,1.0
8,-,-,-,-,-,-,-2.0,-2.0
9,-,-,-,-,-,-,-,-
